In [1]:
import numpy as np
import xgboost as xgb

from constants import MODEL_PARAMS_FOLDER
from template_test import get_param_grid, find_best_params, main_base
from results_processing import review_results, review_params

In [2]:
review_results('xgb_results')
review_params('xgb_params')

left_240x20:	60.67 ± 2.13
left_240x20_fixed:	61.57 ± 1.88
left_240x40:	60.99 ± 1.70
left_240x40_fixed:	61.97 ± 2.13
right_240x20:	59.02 ± 1.88
right_240x20_fixed:	59.11 ± 1.60
right_240x40:	57.98 ± 1.54
right_240x40_fixed:	60.30 ± 1.72
left_240x20:	{'max_depth': 4, 'n_estimators': 80}
left_240x20_fixed:	{'max_depth': 5, 'n_estimators': 100}
left_240x40:	{'max_depth': 4, 'n_estimators': 80}
left_240x40_fixed:	{'max_depth': 2, 'n_estimators': 120}
right_240x20:	{'max_depth': 1, 'n_estimators': 100}
right_240x20_fixed:	{'max_depth': 2, 'n_estimators': 100}
right_240x40:	{'max_depth': 4, 'n_estimators': 20}
right_240x40_fixed:	{'max_depth': 5, 'n_estimators': 100}


In [4]:
review_results('xgb_results_cmim_1')
review_params('xgb_params_cmim_1')

left_240x20:	60.03 ± 1.21
left_240x20_fixed:	60.23 ± 1.52
left_240x40:	59.52 ± 1.49
left_240x40_fixed:	60.18 ± 1.00
right_240x20:	58.72 ± 1.51
right_240x20_fixed:	61.78 ± 2.24
right_240x40:	59.58 ± 1.54
right_240x40_fixed:	61.01 ± 1.17
left_240x20:	{'max_depth': 6, 'n_estimators': 100}
left_240x20_fixed:	{'max_depth': 4, 'n_estimators': 90}
left_240x40:	{'max_depth': 1, 'n_estimators': 100}
left_240x40_fixed:	{'max_depth': 4, 'n_estimators': 60}
right_240x20:	{'max_depth': 1, 'n_estimators': 90}
right_240x20_fixed:	{'max_depth': 4, 'n_estimators': 110}
right_240x40:	{'max_depth': 4, 'n_estimators': 100}
right_240x40_fixed:	{'max_depth': 6, 'n_estimators': 80}


In [2]:
XGB_INIT_PARAMS = {'objective': 'binary:logistic',
                   'use_label_encoder': False,
                   'eval_metric': 'logloss',
                   'tree_method': 'gpu_hist',
                   'subsample': 0.5,
                   'sampling_method': 'gradient_based'}

In [3]:
def get_xgb_param_grid(start_nestimators, step_nestimators, end_nestimators,
                       start_maxdepth, step_maxdepth, end_maxdepth):
    param_grid = get_param_grid(start_param_a=start_nestimators,
                                step_param_a=step_nestimators,
                                end_param_a=end_nestimators,
                                start_param_b=start_maxdepth,
                                step_param_b=step_maxdepth,
                                end_param_b=end_maxdepth,
                                param_a_name='n_estimators',
                                param_b_name='max_depth',
                                param_a_func='linspace',
                                param_b_func='linspace',
                                type_a=int,
                                type_b=int)
    return param_grid


def find_best_xgb_params(train_x: np.ndarray, train_y: np.ndarray,
                         dataset_name: str, partition: int, folder_name: str,
                         pair_method: str, n_jobs: int):
    params = find_best_params(train_x, train_y, dataset_name, partition,
                              folder_name, pair_method,
                              start_param_a=40, step_param_a=20,
                              end_param_a=100, start_param_b=2,
                              step_param_b=2,
                              end_param_b=6,
                              param_a_islog2=False, param_b_islog2=False,
                              param_a_min1=True, param_b_min1=True,
                              param_grid_fn=get_xgb_param_grid,
                              clasif_name='XGB', clasif_fn=xgb.XGBClassifier,
                              n_jobs=n_jobs, init_params=XGB_INIT_PARAMS)
    return params


def main(find_params=True, n_jobs=-1):
    main_base(find_params=find_params,
              out_params_name=MODEL_PARAMS_FOLDER + '/xgb_params',
              find_params_fn=find_best_xgb_params,
              out_results_name='xgb_results',
              clasif_fn=xgb.XGBClassifier,
              use_std_masks=False,
              n_cmim=0,
              n_jobs=n_jobs,
              init_params=XGB_INIT_PARAMS)


def main_cmim(n_cmim: int, find_params=True, n_jobs=-1):
    main_base(find_params=find_params,
              out_params_name=MODEL_PARAMS_FOLDER +
                              f'/xgb_params_cmim_{n_cmim}',
              find_params_fn=find_best_xgb_params,
              out_results_name=f'xgb_results_cmim_{n_cmim}',
              clasif_fn=xgb.XGBClassifier,
              use_std_masks=False,
              n_cmim=n_cmim,
              n_jobs=n_jobs,
              init_params=XGB_INIT_PARAMS)

In [ ]:
main_cmim(1, True, 5)